# Few tests on the integration efficience

In [2]:
### libraries
import sys

sys.path.append('../')

import Velocity_dispersion
import rings2cosmo
import pandas as pd
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import seaborn

from astropy import constants as const
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0 = 67.3, Om0 = 0.315)

In [3]:
numerical = pd.read_csv('results.csv')
numerical.head()

,name,numerical_sigma,integral_result,integral_error,epsabs
0,J2321-0939,240.101777,7446.749998,7.946850e+07,0.149
1,J1106+5228,222.047668,6932.458429,6.784547e+07,0.149
2,J1143-0144,264.867997,10395.782864,9.643616e+07,0.149
3,J0841+3824,230.106668,8225.243472,7.272054e+07,0.149
4,J0044+0113,252.925493,10113.812560,8.628597e+07,0.149


Now I will gerate the analitical values for the same points udes before.

The analitical expresion for $\left \langle  \sigma^2_*\right \rangle$ with constant $\beta$ is
$$
 \left \langle \sigma^2_{LOS,*} \right \rangle =
    \left [ \frac{1}{1 + \gamma}\frac{c^2}{2}\frac{D_s}{D_{ls}}\theta_E \right ]
    \frac{2}{\sqrt{\pi}\lambda(\alpha)(\xi - 2\beta)}
   \frac{\left [ \lambda(\xi)
   - \beta\lambda(\xi + 2) \right ]}{\lambda(\delta)}
   \left(\frac{2\tilde{\sigma}^2_{atm}}{\theta_E^2}\right)^{(2-\alpha)/2} 
   \frac{\Gamma\left ( \frac{3 - \xi}{2} \right )}{\Gamma\left ( \frac{3 - \delta}{2} \right )}
$$


### About pandas
- to filter

``` python

df_n03 = df[df["epsabs"] == 1.49e-03]

```

- to join

``` python
df_n03 = df_n03.set_index('name').join(df_a.set_index('name'))
```

- change the type 
``` python
df_n03 = df_n03.astype('float64')
```

